In [1]:
import ROOT
import os, re, gc, h5py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime
from data_utils import *

Welcome to JupyROOT 6.14/04


In [2]:
copa = np.array([np.zeros((3,3)),np.ones((3,3))]).T

In [3]:
copa[...,1]

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [4]:
n_pols = 9
path_dir = '/mnt/ML-drive/Artem/'
class_names_ft = ['C30keV','C60keV']
class_keys = ['C30keV','C60keV','C100keV','gamma/Cs137','gamma/Co60','fog']
id_header = ['HeaderID','ViewID','GrainID','pol0','pol1','pol2','pol3','pol4','pol5','pol6','pol7','tr_flag','n_pol']
feat_names = ['x','y','z','lx','ly','phi','npx','vol','eps']

feat_array = []
for i in range(9):
    feat_array += [n+str(i) for n in feat_names]
feat_array += ['tr_flag','n_pol']

In [5]:
class_names_ft += ['C100keV/'+i for i in os.listdir(path_dir+'C100keV')] + ['fog/'+i for i in os.listdir(path_dir+'fog')] + ['gamma/Cs137/'+i for i in os.listdir(path_dir+'gamma/Cs137')] + ['gamma/Co60/'+i for i in os.listdir(path_dir+'gamma/Co60')]

In [6]:
class_names_ft

['C30keV',
 'C60keV',
 'C100keV/C2',
 'C100keV/C1',
 'fog/Scan2_Y-4800_1line101views_Re_numofImg200',
 'fog/Scan9_Y-3500_5line505view_numofImage227with9pol',
 'fog/Scan11_Y-2900_6line606view_numofImage227with9pol',
 'fog/Scan8_Y-3800_4line404view_numofImage227with9pol',
 'fog/Scan7_Y-4000_4line404view_numofImage227with9pol',
 'fog/Scan3_Y-4700_1line101views_Re_numofImg200',
 'fog/Scan6_Y-4200_4line404view_numofImage227with9pol',
 'fog/Scan4_Y-4600_2line202view_numofImage227with9pol',
 'gamma/Cs137/400views_2ndrun',
 'gamma/Cs137/400views',
 'gamma/Co60/400views_2ndrun',
 'gamma/Co60/400views']

In [7]:
class_names_C = ['C30keV','C60keV','C100keV/C1','C100keV/C2']

In [8]:
def short_name(name):
    if 'fog' in name: return name.split('_')[0]
    if 'gamma' in name: return name.split('_')[0]+'_'+('2' if '2' in name else '1')
    if 'keV' in name: return 'Carbon/'+name[1:]
    return name

In [9]:
short_name(class_names_C[-1])

'Carbon/100keV/C2'

In [8]:
%%time
pol_ids = {}
for name in class_names_ft:
    pol_ids[name] = pd.read_csv(path_dir+name+'/yandex_bfcl.txt', header=None, names=id_header)
    pol_ids[name] = pol_ids[name].sort_values(by=['HeaderID','GrainID'])

CPU times: user 1.13 s, sys: 110 ms, total: 1.24 s
Wall time: 1.69 s


In [9]:
pol_ids['C60keV'].head(10)

,HeaderID,ViewID,GrainID,pol0,pol1,pol2,pol3,pol4,pol5,pol6,pol7,tr_flag,n_pol
0,1,0,0,8998,9031,9061,9084,9113,-1,-1,-1,-1,5
1,1,0,4,8995,9027,9056,9079,9106,9141,9166,9223,2,8
2,1,0,10,8994,-1,-1,9078,9104,-1,9164,9220,-1,5
3,1,0,12,8993,-1,9053,-1,-1,-1,-1,9219,-1,3
4,1,0,13,8986,9017,9046,9074,9098,9135,9158,9215,-1,8
5,1,0,14,8980,9293,9319,9337,9363,9124,9153,9444,-1,8
6,1,0,25,8651,8685,8717,8751,8784,8813,8844,8908,-2,8
7,1,0,33,9263,9290,9315,9335,9361,-1,-1,9442,2,6
8,1,0,34,9258,9286,-1,-1,-1,-1,-1,-1,-2,2
9,1,0,38,-1,-1,8707,-1,-1,-1,-1,-1,-2,1


* empty polarizations are loaded with zeros! subject to interpolation?

* 9th polarization is a copy of 1st in purpose of cyclicity (periodic boundary conditions)

In [12]:
os.listdir(path_dir+'C100keV/C2/csvs/')[:3]

['284_gr_95_pol_1_cl_381_tr_-2_npol_8.csv',
 '380_gr_351_pol_4_cl_1451_tr_-2_npol_8.csv',
 '466_gr_21_pol_0_cl_3659_tr_-1_npol_8.csv']

In [13]:
%%time
ims_check = []
ims = {}
start = datetime.now()
for name in class_names_C:
    fold = datetime.now()
    ims[name] = load_pol_images(pol_ids[name], path_dir, name, n_pol=9)
    print(short_name(name),' with ',ims[name].shape[0],' images')
    ims_check.append((ims[name][...,0]-ims[name][...,-1]).sum())
    with h5py.File('data_raw_9pol.h5','a') as dfile:
        dfile.create_dataset(short_name(name)+'/images', data=ims[name])
    print('loaded in ',datetime.now()-fold,'\n')
    del ims[name]
    gc.collect()
print('total loading time:',datetime.now()-start)

'load raw images'

Carbon/30keV  with  182179  images
loaded in  3:41:27.964542 

Carbon/60keV  with  189537  images
loaded in  3:50:51.022976 

Carbon/100keV/C1  with  108162  images
loaded in  2:12:25.363611 

Carbon/100keV/C2  with  95683  images
loaded in  1:56:40.556791 

total loading time: 11:41:25.249234
CPU times: user 11h 17min 18s, sys: 8min 41s, total: 11h 25min 59s
Wall time: 11h 41min 25s


In [14]:
ims_check

[0, 0, 0, 0]

In [15]:
%%time
start = datetime.now()
feat_check = []
feat_data = {}
for name in class_names_C:
    fold = datetime.now()
    print('loading features:  ',short_name(name))
    feat_data[name] = get_pol_feat(pol_ids[name], n_pol=9, path_dir=path_dir, class_name=name, feat_names=feat_names)
    feat_check.append((feat_data[name].values[:,:9]-feat_data[name].values[:,-11:-2]).sum(axis=0))
    with h5py.File('data_raw_9pol.h5', 'a') as datafile:
        datafile.create_dataset(short_name(name)+'/features', data=feat_data[name])
    print('\tin ',datetime.now()-fold,'\n')
    del feat_data[name]
    gc.collect()
print('total loading time:',datetime.now()-start)

        
'load features (paralellized automatically)'

loading features:   Carbon/30keV
	in  2:00:45.458735 

loading features:   Carbon/60keV
	in  2:11:31.844276 

loading features:   Carbon/100keV/C1
	in  1:08:18.333568 

loading features:   Carbon/100keV/C2
	in  0:59:13.750377 

total loading time: 6:19:49.527976
CPU times: user 1d 12h 37min 32s, sys: 45min 45s, total: 1d 13h 23min 18s
Wall time: 6h 19min 49s


Warning in <TClass::Init>: no dictionary for class DMRViewHeader is available
Warning in <TClass::Init>: no dictionary for class DMRAffine2D is available
Warning in <TClass::Init>: no dictionary for class DMRCluster is available
Warning in <TClass::Init>: no dictionary for class DMRGrain is available
Warning in <TClass::Init>: no dictionary for class DMRMicrotrack is available
Warning in <TClass::Init>: no dictionary for class DMRImageCl is available
Warning in <TClass::Init>: no dictionary for class DMRImage is available
Warning in <TClass::Init>: no dictionary for class DMRFrame is available


In [16]:
pd.DataFrame(data=np.array(feat_check))

,0,1,2,3,4,5,6,7,8
0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
1,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
2,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN
3,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN


In [17]:
print(feat_names)

['x', 'y', 'z', 'lx', 'ly', 'phi', 'npx', 'vol', 'eps']


In [18]:
with h5py.File('data_raw_9pol.h5','r') as df:
    for name in class_names_C:
        print(name, df[short_name(name)+'/images'].shape,'\t',df[short_name(name)+'/features'].shape)

C30keV (182179, 32, 32, 9) 	 (182179, 83)
C60keV (189537, 32, 32, 9) 	 (189537, 83)
C100keV/C1 (108162, 32, 32, 9) 	 (108162, 83)
C100keV/C2 (95683, 32, 32, 9) 	 (95683, 83)


In [19]:
with h5py.File('data_raw.h5','r') as df:
    for name in class_names_C:
        print(name, df[name+'/images'].shape,'\t',df[name+'/features'].shape)

C30keV (182179, 32, 32, 8) 	 (182179, 74)
C60keV (189537, 32, 32, 8) 	 (189537, 74)
C100keV/C1 (108162, 32, 32, 8) 	 (108162, 74)
C100keV/C2 (95683, 32, 32, 8) 	 (95683, 74)


In [22]:
with h5py.File('data_raw.h5','r') as df:
    copa = df['C30keV/features'][...]
copa[copa[:,-2]<0].shape

(132946, 74)

In [23]:
with h5py.File('data_raw_9pol.h5','r') as df:
    copa = df['Carbon/30keV/features'][...]
copa[copa[:,-2]<0].shape

(171946, 83)

In [31]:
%%time
ims, feat_data = {}, {}
with h5py.File('data_raw_9pol.h5','r') as dfile:
    for name in class_names_ft:
        name = short_name(name)
        ims[name] = dfile[name+'/images'][...]
        feat_data[name] = pd.DataFrame(data=dfile[name+'/features'][...], columns=feat_array)
        print(name,'   \timgs: ',ims[name].shape,'\tfeats: ',feat_data[name].shape)
gc.collect()
print()

Carbon/30keV    	imgs:  (182179, 32, 32, 9) 	feats:  (182179, 83)
Carbon/60keV    	imgs:  (189537, 32, 32, 9) 	feats:  (189537, 83)
Carbon/100keV/C2    	imgs:  (95683, 32, 32, 9) 	feats:  (95683, 83)
Carbon/100keV/C1    	imgs:  (108162, 32, 32, 9) 	feats:  (108162, 83)
fog/Scan2    	imgs:  (6205, 32, 32, 9) 	feats:  (6205, 83)
fog/Scan9    	imgs:  (78928, 32, 32, 9) 	feats:  (78928, 83)
fog/Scan11    	imgs:  (74267, 32, 32, 9) 	feats:  (74267, 83)
fog/Scan8    	imgs:  (64397, 32, 32, 9) 	feats:  (64397, 83)
fog/Scan7    	imgs:  (59160, 32, 32, 9) 	feats:  (59160, 83)
fog/Scan3    	imgs:  (13445, 32, 32, 9) 	feats:  (13445, 83)
fog/Scan6    	imgs:  (52040, 32, 32, 9) 	feats:  (52040, 83)
fog/Scan4    	imgs:  (31263, 32, 32, 9) 	feats:  (31263, 83)
gamma/Cs137/400views_2    	imgs:  (89718, 32, 32, 9) 	feats:  (89718, 83)
gamma/Cs137/400views_1    	imgs:  (87384, 32, 32, 9) 	feats:  (87384, 83)

CPU times: user 55.5 ms, sys: 4.78 s, total: 4.84 s
Wall time: 20.1 s


In [32]:
gc.collect()

44

In [33]:
# for Fog scans
bulk_range = {'2':(-18,24), '9':(-22,21), '11':(-21,22), '8':(-21,22), '7':(-22,22), '3':(-19,17), '6':(-20,23), '4':(-24,23)}
# for Gamma:
# ViewID > 10
# 'bulk' = (-15,10)

In [35]:
list(feat_data.keys())

['Carbon/30keV',
 'Carbon/60keV',
 'Carbon/100keV/C2',
 'Carbon/100keV/C1',
 'fog/Scan2',
 'fog/Scan9',
 'fog/Scan11',
 'fog/Scan8',
 'fog/Scan7',
 'fog/Scan3',
 'fog/Scan6',
 'fog/Scan4',
 'gamma/Cs137/400views_2',
 'gamma/Cs137/400views_1']

In [10]:
for i in range(3):
    class_keys[i] = short_name(class_keys[i])
class_keys

['Carbon/30keV',
 'Carbon/60keV',
 'Carbon/100keV',
 'gamma/Cs137',
 'gamma/Co60',
 'fog']

In [44]:
#%%time
print('Cleaning divergent samples and edge images')
print("Rejecting unphysical dirty emulsion's surfaces for bckg samples")
#print('Splitting into train-val-test (80-10-10) and saving to dataset_phys_clean.h5')
print('')
tmp_name = short_name(class_names_ft[0])
ims['Carbon/100keV'] = np.ones((0, *ims[tmp_name].shape[1:]),dtype=ims[tmp_name].dtype)
ims['fog'] = np.copy(ims['Carbon/100keV'])
ims['gamma/Cs137'] = np.copy(ims['Carbon/100keV'])
feat_data['Carbon/100keV'] = np.ones((0, *feat_data[tmp_name].shape[1:]))
feat_data['fog'] = np.copy(feat_data['Carbon/100keV'])
feat_data['gamma/Cs137'] = np.copy(feat_data['Carbon/100keV'])

bads = {}
start = datetime.now()
for name in class_names_ft:
    s_name = short_name(name)
    fold = datetime.now()
    bads[name] = set()
    if 'fog' in name or 'gamma' in name:
        tmp_range = bulk_range[name[8:].split('_')[0]] if 'fog' in name else (-15,10)
        for i in range(8):
            bads[name] |= set(feat_data[s_name][feat_data[s_name]['z'+str(i)]<tmp_range[0]].index)
            bads[name] |= set(feat_data[s_name][feat_data[s_name]['z'+str(i)]>tmp_range[1]].index)
    if 'gamma' in name: bads[name] |= set(feat_data[s_name][pol_ids[name]['ViewID']<10].index)
    bads[name] = bad_inds(pol_ids[name], imgs=ims[s_name], features=feat_data[s_name],
                          isolated=True, quant=0.999, bad_i=bads[name])
    print(len(bads[name]),'\tBad ',s_name,' cleaned in ',datetime.now()-fold)
    np.savetxt('bad_sample_ids/phys_'+'_'.join(s_name.split('/'))+'.txt', bads[name], fmt='%d')
    mask = np.ones(pol_ids[name].shape[0], dtype=bool)
    mask[bads[name]] = False
    if 'fog' in name:
        ims['fog'] = np.vstack((ims['fog'],ims[s_name][mask]))
        feat_data['fog'] = np.vstack((feat_data['fog'],(feat_data[s_name].values)[mask]))
    elif 'gamma' in name:
        ims['gamma/Cs137'] = np.vstack((ims['gamma/Cs137'],ims[s_name][mask]))
        feat_data['gamma/Cs137'] = np.vstack((feat_data['gamma/Cs137'],(feat_data[s_name].values)[mask]))
    elif '100keV' in name:
        ims['Carbon/100keV'] = np.vstack((ims['Carbon/100keV'],ims[s_name][mask]))
        feat_data['Carbon/100keV'] = np.vstack((feat_data['Carbon/100keV'],(feat_data[s_name].values)[mask]))
    else:
        ims[s_name] = ims[s_name][mask]
        feat_data[s_name] = (feat_data[s_name].values)[mask]
        feat_data[s_name] = pd.DataFrame(data=feat_data[s_name], columns=feat_array)
    _=gc.collect()
feat_data['fog'] = pd.DataFrame(data=feat_data['fog'], columns=feat_array)
feat_data['gamma/Cs137'] = pd.DataFrame(data=feat_data['gamma/Cs137'], columns=feat_array)
feat_data['Carbon/100keV'] = pd.DataFrame(data=feat_data['Carbon/100keV'], columns=feat_array)
print('Total cleaning time:',datetime.now()-start)

for name in class_keys:
    im_tr, im_val = train_test_split(ims[name], test_size=0.4, shuffle=False)
    feat_tr, feat_val = train_test_split(feat_data[name].values, test_size=0.4, shuffle=False)
    
    with h5py.File('dataset_phys_clean_9pol.h5', 'a') as datafile:
        datafile.create_dataset(name+'/images/train', data=im_tr)
        datafile.create_dataset(name+'/images/val', data=im_val)
        datafile.create_dataset(name+'/features/train', data=feat_tr)
        datafile.create_dataset(name+'/features/val', data=feat_val)

_='''print('Wall time ~4 min\n')
for name in class_names_ft:
    f_name = '_'.join(name.split('/'))
    print(np.loadtxt('bad_sample_ids/'+f_name+'.txt').shape[0], '\tBad ',f_name)'''

Cleaning divergent samples and edge images
Rejecting unphysical dirty emulsion's surfaces for bckg samples

12951 	Bad  Carbon/30keV  cleaned in  0:00:50.204328
16002 	Bad  Carbon/60keV  cleaned in  0:00:52.200815
11026 	Bad  Carbon/100keV/C2  cleaned in  0:00:26.594517
11811 	Bad  Carbon/100keV/C1  cleaned in  0:00:29.950821
2404 	Bad  fog/Scan2  cleaned in  0:00:01.693442
61027 	Bad  fog/Scan9  cleaned in  0:00:22.314968
54063 	Bad  fog/Scan11  cleaned in  0:00:20.758369
49573 	Bad  fog/Scan8  cleaned in  0:00:18.057152
46196 	Bad  fog/Scan7  cleaned in  0:00:16.414714
9950 	Bad  fog/Scan3  cleaned in  0:00:03.732599
37657 	Bad  fog/Scan6  cleaned in  0:00:14.436520
23883 	Bad  fog/Scan4  cleaned in  0:00:08.719270
41094 	Bad  gamma/Cs137/400views_2  cleaned in  0:00:25.167427
37840 	Bad  gamma/Cs137/400views_1  cleaned in  0:00:23.685827
Total cleaning time: 0:05:19.444428


In [11]:
with h5py.File('dataset_phys_clean_9pol.h5', 'r') as dfile:
    for name in class_keys:
        print(name+':')
        for i in ['images','features']:
            print(i+':')
            for t in ['train','val']:
                pr = ' \t' if i=='images' else '\t\t'
                print(t+': ', dfile[name+'/'+i+'/'+t].shape, end=pr)
            print('',end='\n')
        print('',end='\n')

Carbon/30keV:
images:
train:  (101536, 32, 32, 9) 	val:  (67692, 32, 32, 9) 	
features:
train:  (101536, 83)		val:  (67692, 83)		

Carbon/60keV:
images:
train:  (104121, 32, 32, 9) 	val:  (69414, 32, 32, 9) 	
features:
train:  (104121, 83)		val:  (69414, 83)		

Carbon/100keV:
images:
train:  (108604, 32, 32, 9) 	val:  (72404, 32, 32, 9) 	
features:
train:  (108604, 83)		val:  (72404, 83)		

gamma/Cs137:
images:
train:  (58900, 32, 32, 9) 	val:  (39268, 32, 32, 9) 	
features:
train:  (58900, 83)		val:  (39268, 83)		

gamma/Co60:
images:
train:  (69870, 32, 32, 9) 	val:  (46580, 32, 32, 9) 	
features:
train:  (69870, 83)		val:  (46580, 83)		

fog:
images:
train:  (56971, 32, 32, 9) 	val:  (37981, 32, 32, 9) 	
features:
train:  (56971, 83)		val:  (37981, 83)		



### Producing ready-to-use datasets to feed into train-test

In [12]:
readir = 'ready-to-float/'
dset = '/mnt/ML-drive/Artem/Python/NEWS/data/dataset_phys_clean_9pol.h5'
bckg_names = ['fog']
with h5py.File(dset,'r') as df:
    sig_names = ['Carbon/'+s for s in df['Carbon'].keys()]
    bckg_names += ['gamma/'+s for s in df['gamma'].keys()]

In [13]:
ims, feats = {},{}
with h5py.File('dataset_phys_clean_9pol.h5', 'r') as dfile:
    for name in class_keys:
        for i in ['images','features']:
            for t in ['train','val']:
                if i=='images': ims[name+'/'+i+'/'+t] = dfile[name+'/'+i+'/'+t][...]
                else: feats[name+'/'+i+'/'+t] = dfile[name+'/'+i+'/'+t][...]

In [15]:
%%time
for s in sig_names:
    s_ = '-'.join(s.split('/'))
    for b in bckg_names:
        b_ = '-'.join(b.split('/'))
        with h5py.File(readir+s_+'_'+b_+'.h5','a') as sb_file:
            for t in ['train','val']:
                X_im = np.vstack((ims[s+'/images/'+t],ims[b+'/images/'+t]))
                X_f = np.vstack((feats[s+'/features/'+t],feats[b+'/features/'+t]))
                y = np.append(np.ones(ims[s+'/images/'+t].shape[0]),np.zeros(ims[b+'/images/'+t].shape[0]))
                shuff = np.random.permutation(len(y))
                sb_file.create_dataset(t+'/images', data=X_im[shuff], dtype=np.float32)
                sb_file.create_dataset(t+'/features', data=X_f[shuff], dtype=np.float32)
                sb_file.create_dataset(t+'/labels', data=y[shuff], dtype=int)

CPU times: user 1min 12s, sys: 51.6 s, total: 2min 3s
Wall time: 12min 6s


* Example of loading the libDMRoot in PyROOT

In [46]:
f = ROOT.TFile.Open(path_dir+'C60keV/dm_tracks.dm.root','read')

Warning in <TClass::Init>: no dictionary for class DMRViewHeader is available
Warning in <TClass::Init>: no dictionary for class DMRAffine2D is available
Warning in <TClass::Init>: no dictionary for class DMRCluster is available
Warning in <TClass::Init>: no dictionary for class DMRGrain is available
Warning in <TClass::Init>: no dictionary for class DMRMicrotrack is available
Warning in <TClass::Init>: no dictionary for class DMRImageCl is available
Warning in <TClass::Init>: no dictionary for class DMRImage is available
Warning in <TClass::Init>: no dictionary for class DMRFrame is available


In [47]:
ROOT.gSystem.Load('libDMRoot')

0

Error in <TCling::RegisterModule>: cannot find dictionary module DMRootCint_rdict.pcm


In [48]:
arun = ROOT.DMRRun("dm_tracks.dm.root")

DMRRun::OpenNew : Open new file dm_tracks.dm.root


Warning in <TTree::Bronch>: Using split mode on a class: DMRCluster with a custom Streamer
Warning in <TTree::Bronch>: Using split mode on a class: DMRGrain with a custom Streamer
Warning in <TTree::Bronch>: Using split mode on a class: DMRMicrotrack with a custom Streamer
Warning in <TTree::Bronch>: Using split mode on a class: DMRImageCl with a custom Streamer
Warning in <TTree::Bronch>: Using split mode on a class: DMRFrame with a custom Streamer
